In [99]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import gc

patterns_sept_1 = "./data/patterns-part1.csv.gz"
spark = SparkSession.builder.appName("patterns_sept").getOrCreate()
patterns_df = spark.read.csv(patterns_sept_1,header= True)

In [63]:
# Print configuration basics
app_name = spark.conf.get('spark.app.name')
driver_tcp_port = spark.conf.get('spark.driver.port')
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions)

Name: patterns_sept
Driver TCP port: 65395
Number of partitions: 200


In [66]:
patterns_df.printSchema()

root
 |-- safegraph_place_id: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- street_address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- safegraph_brand_ids: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- raw_visit_counts: string (nullable = true)
 |-- raw_visitor_counts: string (nullable = true)
 |-- visits_by_day: string (nullable = true)
 |-- poi_cbg: string (nullable = true)
 |-- visitor_home_cbgs: string (nullable = true)
 |-- visitor_daytime_cbgs: string (nullable = true)
 |-- visitor_work_cbgs: string (nullable = true)
 |-- visitor_country_of_origin: string (nullable = true)
 |-- distance_from_home: string (nullable = true)
 |-- median_dwell: string (nullable = true)
 |-- bucketed_dwell_times: string (nullable = true)
 |-- related_same_d

In [67]:
patterns_df.createOrReplaceTempView('patterns_view')

In [68]:
spark.catalog.listTables()

[Table(name='patterns_view', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [69]:
traffic = patterns_df.filter(patterns_df.raw_visit_counts > 5)

In [70]:
query = "SELECT * FROM patterns_view WHERE location_name IS NOT NULL"

In [90]:
dataframe = spark.sql(query)

In [91]:
dataframe.show(n=20,truncate=False,vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------
 safegraph_place_id        | sg:0041bdf50f984826acd68ce19922ecd4                                                        
 location_name             | Sullivan Chris M Dr                                                                        
 street_address            | 403 Eighth St                                                                              
 city                      | Winnsboro                                                                                  
 region                    | LA                                                                                         
 postal_code               | 71295                                                                                      
 safegraph_brand_ids       | null                                                                                       
 brands                    | nul

In [92]:
%%time
patterns_df.count()

CPU times: user 1.83 ms, sys: 1.75 ms, total: 3.58 ms
Wall time: 10.2 s


1063201

In [93]:
import pandas as pd

In [98]:
patterns_pandas_df = pd.read_csv("./data/patterns-part1.csv.gz")

In [95]:
patterns_pandas_df.head(5)

,safegraph_place_id,location_name,street_address,city,region,postal_code,safegraph_brand_ids,brands,date_range_start,date_range_end,...,visitor_work_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type
0,sg:0041bdf50f984826acd68ce19922ecd4,Sullivan Chris M Dr,403 Eighth St,Winnsboro,LA,71295,NaN,NaN,2020-06-01T00:00:00-05:00,2020-07-01T00:00:00-05:00,...,"{""220839706003"":5,""220419503001"":4}","{""US"":38}",4359.0,31.5,"{""<5"":3,""5-20"":39,""21-60"":19,""61-240"":24,"">240...","{""Tiger Rock Martial Arts"":50,""Shoe Dept Encor...","{""Walmart"":65,""Chevron"":57,""Shell Oil"":55,""Son...","[16,16,13,11,11,10,11,11,13,12,15,15,13,16,17,...","{""Monday"":13,""Tuesday"":9,""Wednesday"":7,""Thursd...","{""android"":15,""ios"":21}"
1,sg:005761d9681947629bf02a64cfd4aec2,U Pick Carlsbad Strawberry Co,1000 Cannon Rd,Carlsbad,CA,92008,NaN,NaN,2020-06-01T00:00:00-07:00,2020-07-01T00:00:00-07:00,...,"{""060730101071"":5,""060730041004"":4,""0603722600...","{""US"":382,""MX"":7}",50514.0,10.0,"{""<5"":20,""5-20"":278,""21-60"":86,""61-240"":26,"">2...","{""In-N-Out Burger"":12,""GMC (General Motors Com...","{""Starbucks"":49,""Target"":32,""Costco Wholesale ...","[0,0,0,0,0,0,3,2,2,13,46,64,82,85,82,84,52,18,...","{""Monday"":48,""Tuesday"":34,""Wednesday"":39,""Thur...","{""android"":131,""ios"":264}"
2,sg:007236f9d77d4158bbd4d55f39d98e49,CJ's Auto Glass Repair,190 State Route 18,East Brunswick,NJ,8816,NaN,NaN,2020-06-01T00:00:00-04:00,2020-07-01T00:00:00-04:00,...,"{""340350542022"":4,""340350535011"":4,""3402300640...","{""US"":64}",11440.0,44.5,"{""<5"":2,""5-20"":21,""21-60"":38,""61-240"":36,"">240...","{""Wawa"":28}","{""Dunkin'"":43,""The Home Depot"":28,""ShopRite"":2...","[0,0,0,0,1,2,2,4,15,32,38,35,18,9,10,5,8,7,3,1...","{""Monday"":31,""Tuesday"":4,""Wednesday"":24,""Thurs...","{""android"":31,""ios"":38}"
3,sg:0080800153224074974597d48f0a801b,La Jolla Open Aire Farmer's Market,7300 Girard Ave La Jolla Elementary School,La Jolla,CA,92037,NaN,NaN,2020-06-01T00:00:00-07:00,2020-07-01T00:00:00-07:00,...,"{""060730081012"":4,""060730083391"":4}","{""US"":42}",3850.0,22.5,"{""<5"":0,""5-20"":26,""21-60"":21,""61-240"":15,"">240...","{""Primos"":100,""Starbucks"":98,""Costco Wholesale...","{""Vons"":36,""Chevron"":35,""Starbucks"":35,""Westfi...","[0,0,0,0,0,1,2,5,10,11,14,23,15,13,12,10,4,2,3...","{""Monday"":18,""Tuesday"":10,""Wednesday"":11,""Thur...","{""android"":10,""ios"":38}"
4,sg:009679e673aa46c0aae9c01e7fa7ba3d,The Spot Cafe,628 7th St,Rockford,IL,61104,NaN,NaN,2020-06-01T00:00:00-05:00,2020-07-01T00:00:00-05:00,...,"{""172010011001"":4,""172010018003"":4,""1720100120...","{""US"":35}",3543.0,29.0,"{""<5"":4,""5-20"":28,""21-60"":15,""61-240"":21,"">240...","{""Blain’s Farm & Fleet"":50,""Dollar Tree"":49,""O...","{""Mobil"":44,""Fas Mart"":35,""BP"":31,""Walgreens"":...","[11,13,12,12,11,12,12,11,12,12,14,20,19,17,14,...","{""Monday"":13,""Tuesday"":11,""Wednesday"":17,""Thur...","{""android"":28,""ios"":16}"


In [96]:
%%time
patterns_pandas_df.count()

CPU times: user 927 ms, sys: 53.9 ms, total: 981 ms
Wall time: 1e+03 ms


safegraph_place_id           1063201
location_name                1063201
street_address               1063201
city                         1063201
region                       1063201
postal_code                  1063201
safegraph_brand_ids           233703
brands                        233703
date_range_start             1063201
date_range_end               1063201
raw_visit_counts             1063201
raw_visitor_counts           1063201
visits_by_day                1063201
poi_cbg                      1063196
visitor_home_cbgs            1063201
visitor_daytime_cbgs         1063201
visitor_work_cbgs            1063201
visitor_country_of_origin    1056639
distance_from_home            903767
median_dwell                 1063201
bucketed_dwell_times         1063201
related_same_day_brand       1063201
related_same_month_brand     1063201
popularity_by_hour           1063201
popularity_by_day            1063201
device_type                  1063201
dtype: int64

DataFrame[safegraph_place_id: string, location_name: string, street_address: string, city: string, region: string, postal_code: string, safegraph_brand_ids: string, brands: string, date_range_start: string, date_range_end: string, raw_visit_counts: string, raw_visitor_counts: string, visits_by_day: string, poi_cbg: string, visitor_home_cbgs: string, visitor_daytime_cbgs: string, visitor_work_cbgs: string, visitor_country_of_origin: string, distance_from_home: string, median_dwell: string, bucketed_dwell_times: string, related_same_day_brand: string, related_same_month_brand: string, popularity_by_hour: string, popularity_by_day: string, device_type: string]

In [93]:
dataframe_fast.is_cached

True

DataFrame[safegraph_place_id: string, location_name: string, street_address: string, city: string, region: string, postal_code: string, safegraph_brand_ids: string, brands: string, date_range_start: string, date_range_end: string, raw_visit_counts: string, raw_visitor_counts: string, visits_by_day: string, poi_cbg: string, visitor_home_cbgs: string, visitor_daytime_cbgs: string, visitor_work_cbgs: string, visitor_country_of_origin: string, distance_from_home: string, median_dwell: string, bucketed_dwell_times: string, related_same_day_brand: string, related_same_month_brand: string, popularity_by_hour: string, popularity_by_day: string, device_type: string]

False

'200'

'56068'

'patterns_sept'

== Physical Plan ==
*(1) Project [safegraph_place_id#2055, location_name#2056, street_address#2057, city#2058, region#2059, postal_code#2060, safegraph_brand_ids#2061, brands#2062, date_range_start#2063, date_range_end#2064, raw_visit_counts#2065, raw_visitor_counts#2066, visits_by_day#2067, poi_cbg#2068, visitor_home_cbgs#2069, visitor_daytime_cbgs#2070, visitor_work_cbgs#2071, visitor_country_of_origin#2072, distance_from_home#2073, median_dwell#2074, bucketed_dwell_times#2075, related_same_day_brand#2076, related_same_month_brand#2077, popularity_by_hour#2078, ... 2 more fields]
+- *(1) Filter isnotnull(location_name#2056)
   +- *(1) FileScan csv [safegraph_place_id#2055,location_name#2056,street_address#2057,city#2058,region#2059,postal_code#2060,safegraph_brand_ids#2061,brands#2062,date_range_start#2063,date_range_end#2064,raw_visit_counts#2065,raw_visitor_counts#2066,visits_by_day#2067,poi_cbg#2068,visitor_home_cbgs#2069,visitor_daytime_cbgs#2070,visitor_work_cbgs#2071,visitor_co